# PRACTICA OBLIGATORIA: Transformación y limpieza
La práctica obligatoria de esta unidad consiste en cargar un dataset de Netflix y hacer un repaso completo del mismo, limpiando, transformando y creando algunas variables. Descarga este notebook en tu ordenador y trabaja en local. Ten en cuenta que tendrás que descar los directorios de imágenes y datos adicionales, si los hubiera.

* Recuerda que debes subirla a tu repositorio personal antes de la sesión en vivo para que puntúe adecuadamente.
* Recuerda también que no es necesario que esté perfecta, sólo es necesario que se vea el esfuerzo.
* Esta práctica se resolverá en la sesión en vivo correspondiente y la solución se publicará en el repo del curso.

### 0 Carga de datos y primera exploración

In [2]:
import numpy as np
import pandas as pd

df_peliculas = pd.read_csv("C:/Users/david/Downloads/dataset_netflix_titles.csv")

In [3]:
df_peliculas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


### 1 Duplicados y cardinalidad
1. ¿Existen filas duplicadas?¿Cuánta

In [5]:
num_duplicados = df_peliculas.duplicated().sum()
print(f"Número de filas duplicadas: {num_duplicados}")

Número de filas duplicadas: 0


2. Deshazte de las filas duplicadas, si las hubiera. Quédate con las últimas copias.

In [10]:
df_peliculas = df_peliculas.drop_duplicates(keep='last')
num_duplicados_restantes = df_peliculas.duplicated().sum()
print(f"Duplicados restantes: {num_duplicados_restantes}")

Duplicados restantes: 0


3. Ahora que no hay duplicados, tiene sentido calcular la cardinalidad de cada columna. Haz un programa que la obtenga y guarde el resultado en un diccionario con claves el nombre de las columnas y valores la cardinalidad (tal y como la hemos visto en el workout) de cada columna. Nota: ¿has tenido en cuenta los valores nulos?

In [28]:
cardinalidad = {col: df_peliculas[col].nunique(dropna=True) for col in df.columns}
print(cardinalidad)

{'show_id': 8807, 'type': 5, 'title': 8806, 'director': 4528, 'cast': 7692, 'country': 748, 'date_added': 1714, 'release_year': 74, 'rating': 17, 'duration': 220, 'listed_in': 514, 'description': 8775}


4. Haz un pequeño programa que recorra el diccionario y nos diga si hay alguna columna que pueda ser un buen índice (cardinalidad = 100%)

In [13]:
total_filas = len(df_peliculas)
posibles_indices = [col for col, card in cardinalidad.items() if card == total_filas]
if posibles_indices:
    print("Las siguientes columnas pueden ser un buen índice:")
    print(posibles_indices)
else:
    print("No hay columnas con cardinalidad del 100% que puedan ser un buen índice.")

Las siguientes columnas pueden ser un buen índice:
['show_id', 'title']


### 2 Limpieza y transformación (I)

El primer paso en la limpieza es intentar detectar que campos están "sucios", una posible guía podría ser, encontrar:

* Campos numéricos inconsistentes (el típico campo con la coma y el punto decimal mezclado, o con formatos numéricos combinados)
* Campos con valores inconsistentes (por ejemplo aparecen valores que no tienen relación con la columna en la que están porque están mal colocados, porque se han incluido por error, valores medidos en una unidad frente a otros medidos en otra)
* Campos con valores que son iguales pero están representados de formas diferentes (por ejemplo un actor que unas veces aparece con el nombre en un orden y otras veces con el nombre en otro, o el nombre de un país como USA que aparezca como EEUU otras veces, o cadenas que tienen espacios al principio y al final y Python las considera diferentes: " Clint Eastwood " y "Clint Eastwood").

Vamos a hacer un pequeño análisis siguiendo las pautas anteriores pero sin profundizar en ellas, en tu EDA sí que tendrás que hacerlo. Para ello nos vamos a centrar en los campos con cardinalidad menor que un 10% (el resto requiere un análsis mucho más exahustivo que se nos va fuera del alcance de la práctica, pero, repito en tu EDA tendrás que hacerlo)

In [21]:
import numpy as np
for col in df_peliculas.select_dtypes(include=['object']).columns:
    df_peliculas[col] = df_peliculas[col].str.strip()

def detectar_formato_numerico(df_peliculas):
    columnas_potencialmente_erroneas = []
    for col in df_peliculas.columns:
        if df[col].dtype == 'object':  
            try:
                df_peliculas[col].astype(float)  
            except ValueError:
                columnas_potencialmente_erroneas.append(col)
    return columnas_potencialmente_erroneas

def detectar_valores_atipicos(df_peliculas):
    valores_atipicos = {}
    for col in df_peliculas.select_dtypes(include=['object']).columns:
        valores_unicos = df_peliculas[col].unique()
        if len(valores_unicos) > 20:
            continue
        valores_atipicos[col] = valores_unicos
    return valores_atipicos

def detectar_variantes_texto(df):
    variantes_detectadas = {}
    for col in df.select_dtypes(include=['object']).columns:
        valores_unicos = df_peliculas[col].dropna().unique()
        valores_normalizados = set(v.strip().lower().replace(".", "").replace(",", "") for v in valores_unicos)
        if len(valores_unicos) != len(valores_normalizados):  # Si hay diferencias, hay inconsistencias
            variantes_detectadas[col] = list(valores_unicos)
    return variantes_detectadas

columnas_numericas_inconsistentes = detectar_formato_numerico(df_peliculas)
valores_atipicos = detectar_valores_atipicos(df_peliculas)
variantes_texto = detectar_variantes_texto(df_peliculas)

print("Columnas con formatos numéricos inconsistentes:", columnas_numericas_inconsistentes)
print("\nPosibles valores atípicos en columnas categóricas:", valores_atipicos)
print("\nColumnas con variantes textuales de un mismo valor:", variantes_texto)

 
    


Columnas con formatos numéricos inconsistentes: ['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added', 'rating', 'duration', 'listed_in', 'description']

Posibles valores atípicos en columnas categóricas: {'type': array(['Movie', 'TV Show', 'Film', 'Show', 'Series TV'], dtype=object), 'rating': array(['PG-13', 'TV-MA', 'PG', 'TV-14', 'TV-PG', 'TV-Y', 'TV-Y7', 'R',
       'TV-G', 'G', 'NC-17', '74 min', '84 min', '66 min', 'NR', nan,
       'TV-Y7-FV', 'UR'], dtype=object)}

Columnas con variantes textuales de un mismo valor: {'title': ['Dick Johnson Is Dead', 'Blood & Water', 'Ganglands', 'Jailbirds New Orleans', 'Kota Factory', 'Midnight Mass', 'My Little Pony: A New Generation', 'Sankofa', 'The Great British Baking Show', 'The Starling', 'Vendetta: Truth, Lies and The Mafia', 'Bangkok Breaking', 'Je Suis Karl', 'Confessions of an Invisible Girl', 'Crime Stories: India Detectives', 'Dear White People', "Europe's Most Dangerous Man: Otto Skorzeny in Spain", 'Falsa id

In [22]:
df_peliculas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


2. Lista los campos con cardinalidad menor a 10%

In [23]:
umbral = 0.1 * len(df)
columnas_baja_cardinalidad = {col: card for col, card in cardinalidad.items() if card < umbral}
print("Columnas con cardinalidad menor al 10%:")
for col, card in columnas_baja_cardinalidad.items():
    print(f"{col}: {card} valores únicos")


Columnas con cardinalidad menor al 10%:
type: 5 valores únicos
country: 748 valores únicos
release_year: 74 valores únicos
rating: 17 valores únicos
duration: 220 valores únicos
listed_in: 514 valores únicos


4. Limpia o simplifica/transforma dos de los campos detectados en 3, si hay al menos dos, si no, uno. Para ello puedes:
* 4.1 Cualquier agrupa los valores de los campos tipo categoría con categorías similares que se llamen distinto.
* 4.2 Quita valores extraños o que no correspondan. Conviertelos a "", y luego busca un valor para sustituirlos.

In [27]:
def agrupar_y_limpieza(df_peliculas, column, agrupamientos):
    for valor_original, valor_normalizado in agrupamientos.items():
        df_peliculas[column] = df_peliculas[column].replace(valor_original, valor_normalizado)

    valores_validos = set(agrupamientos.values()) 
    df_peliculas[column] = df_peliculas[column].apply(lambda x: x if x in valores_validos else "")
    
    return df

agrupamientos_1 = {
    "USA": "Estados Unidos",
    "EEUU": "Estados Unidos",
    "U.S.": "Estados Unidos",
    "España": "España",
    "Espana": "España"
}

agrupamientos_2 = {
    "Horror": "Terror",
    "thriller": "Terror",
    "comedy": "Comedia",
    "drama": "Drama"
}

columnas_variantes = list(variantes_texto.keys())[:2] 
for col in columnas_variantes:
    if col in agrupamientos_1:
        df = agrupar_y_limpieza(df, col, agrupamientos_1)
    elif col in agrupamientos_2:
        df = agrupar_y_limpieza(df, col, agrupamientos_2)

print("Primeras filas después de agrupar y limpiar:")
print(df_peliculas.head())
df_peliculas


Primeras filas después de agrupar y limpiar:
  show_id     type                  title         director  \
0      s1    Movie   Dick Johnson Is Dead  Kirsten Johnson   
1      s2  TV Show          Blood & Water              NaN   
2      s3  TV Show              Ganglands  Julien Leclercq   
3      s4  TV Show  Jailbirds New Orleans              NaN   
4      s5  TV Show           Kota Factory              NaN   

                                                cast        country  \
0                                                NaN  United States   
1  Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...   South Africa   
2  Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...            NaN   
3                                                NaN            NaN   
4  Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...          India   

           date_added  release_year rating   duration  \
0  September 25, 2021          2020  PG-13     90 min   
1  September 24, 2021          2021  TV-M

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8803,s8804,TV Show,Zombie Dumb,NaN,NaN,NaN,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g..."
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."
